# Optimization of Noise Generator

We want to find the right Hyperparameters for the Noise Generator.
For the Optimization, we will use the `Optuna` Framework

The correct Hyperparameters for the Noise will be found for:
- `GeFeU`
- `GEMU`

In [1]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import random

import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.join('..', 'src'))

import gefeu
import gemu
import mlp_dataclass
import metrics
from helper import load_models_dict

___
## GeFeU

In [2]:
import logging
import sys
import pickle
import optuna
import os

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study1_name = "GenOptiGeFeU"  # Unique identifier of the study.
storage1_name = "sqlite:///{}.db".format(study1_name)

if os.path.exists("sampler_gefeu.pkl"):
    restored1_sampler = pickle.load(open("sampler_gefeu.pkl", "rb"))
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True, sampler=restored1_sampler)
else:
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True)

[I 2025-01-08 17:26:36,769] Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.


Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.


In [3]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    # print(f"Epochs: {opt_Epochs} 
    #       |\nLearning Rate: {opt_Learning_Rate} 
    #       |\nBatch Size: {opt_Batch_Size} 
    #       |\nN2R Ratio: {opt_N2R_Ratio} 
    #       |\nRegularization Term: {opt_Regularization_term} 
    #       |\nNoise Dim: {opt_Noise_Dim}
    #       |\nImpair LR: {opt_Impair_LR}
    #       |\nRepair LR: {opt_Repair_LR}")

    l1 = trial.suggest_int('l1', 32, 1024)
    l2 = trial.suggest_int('l2', 32, 1024)
    l3 = trial.suggest_int('l3', 32, 1024)
    l4 = trial.suggest_int('l4', 32, 1024)
    l5 = trial.suggest_int('l5', 32, 1024)
    l6 = trial.suggest_int('l6', 32, 1024)
    l7 = trial.suggest_int('l7', 32, 1024)
    l8 = trial.suggest_int('l8', 32, 1024)
    l9 = trial.suggest_int('l9', 32, 1024)
    n_layers = trial.suggest_int('n_layers', 1, 9)

    Layers = [l1, l2, l3, l4, l5, l6, l7, l8, l9]
    Layers = Layers[:n_layers]
    # print("Layers: ", Layers)

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]
    
    unlearned = gefeu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = unlearned,
        model2 = exact,
        data_loader = valid_dl,
    )

    return div

study_gefeu.optimize(objective, n_trials=3)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gefeu.pkl", "wb") as fout:
    pickle.dump(study_gefeu.sampler, fout)

[W 2025-01-08 17:29:13,447] Trial 9 failed with parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.23748483203027362, 'opt_Batch_Size': 65, 'opt_N2R_Ratio': 19.38557923428422, 'opt_Regularization_term': 0.24958432292577604, 'opt_Noise_Dim': 160, 'opt_Impair_LR': 0.25698894507017267, 'opt_Repair_LR': 0.146004795612318, 'l1': 157, 'l2': 408, 'l3': 801, 'l4': 439, 'l5': 999, 'l6': 982, 'l7': 358, 'l8': 938, 'l9': 259, 'n_layers': 9} because of the following error: The value nan is not acceptable.


Trial 9 failed with parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.23748483203027362, 'opt_Batch_Size': 65, 'opt_N2R_Ratio': 19.38557923428422, 'opt_Regularization_term': 0.24958432292577604, 'opt_Noise_Dim': 160, 'opt_Impair_LR': 0.25698894507017267, 'opt_Repair_LR': 0.146004795612318, 'l1': 157, 'l2': 408, 'l3': 801, 'l4': 439, 'l5': 999, 'l6': 982, 'l7': 358, 'l8': 938, 'l9': 259, 'n_layers': 9} because of the following error: The value nan is not acceptable.


[W 2025-01-08 17:29:13,447] Trial 9 failed with value nan.


Trial 9 failed with value nan.


[W 2025-01-08 17:30:39,494] Trial 10 failed with parameters: {'opt_Epochs': 2, 'opt_Learning_Rate': 0.21294084437549102, 'opt_Batch_Size': 151, 'opt_N2R_Ratio': 16.620943314445984, 'opt_Regularization_term': 0.07861805166230335, 'opt_Noise_Dim': 233, 'opt_Impair_LR': 0.18494713160029108, 'opt_Repair_LR': 0.20806078762507987, 'l1': 700, 'l2': 47, 'l3': 34, 'l4': 103, 'l5': 782, 'l6': 856, 'l7': 722, 'l8': 273, 'l9': 938, 'n_layers': 9} because of the following error: The value nan is not acceptable.


...

Trial 10 failed with value nan.


[W 2025-01-08 17:31:09,363] Trial 11 failed with parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.031038515637847423, 'opt_Batch_Size': 115, 'opt_N2R_Ratio': 4.176981079294669, 'opt_Regularization_term': 0.16913230505811241, 'opt_Noise_Dim': 404, 'opt_Impair_LR': 0.15078688621431519, 'opt_Repair_LR': 0.022262680830003136, 'l1': 382, 'l2': 757, 'l3': 401, 'l4': 169, 'l5': 914, 'l6': 506, 'l7': 839, 'l8': 92, 'l9': 746, 'n_layers': 6} because of the following error: The value nan is not acceptable.


Trial 11 failed with parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.031038515637847423, 'opt_Batch_Size': 115, 'opt_N2R_Ratio': 4.176981079294669, 'opt_Regularization_term': 0.16913230505811241, 'opt_Noise_Dim': 404, 'opt_Impair_LR': 0.15078688621431519, 'opt_Repair_LR': 0.022262680830003136, 'l1': 382, 'l2': 757, 'l3': 401, 'l4': 169, 'l5': 914, 'l6': 506, 'l7': 839, 'l8': 92, 'l9': 746, 'n_layers': 6} because of the following error: The value nan is not acceptable.


[W 2025-01-08 17:31:09,364] Trial 11 failed with value nan.


Trial 11 failed with value nan.


## Standard Values

In [ ]:
# standard_model = src.gefeu._main(
#     model=None,
#     dataset_name="mnist",
#     t_Epochs=,
#     t_Batch_Size=,
#     t_Learning_Rate=,
#     t_N2R_Ratio=,
#     t_Regularization_term=,
#     t_Layers=,
#     t_Noise_Dim=,
#     t_Impair_LR=,
#     t_Repair_LR=,
#     logs=True,
#     model_eval_logs=False,
# )

___
## GEMU

In [ ]:
# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study2_name = "GenOptiGEMU"  # Unique identifier of the study.
storage2_name = "sqlite:///{}.db".format(study2_name)

if os.path.exists("sampler_gemu.pkl"):
    restored2_sampler = pickle.load(open("sampler_gemu.pkl", "rb"))
study_gemu = optuna.create_study(study_name=study2_name, storage=storage2_name, load_if_exists=True, sampler=restored2_sampler)

In [ ]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    # print(f"Epochs: {opt_Epochs} 
    #       |\nLearning Rate: {opt_Learning_Rate} 
    #       |\nBatch Size: {opt_Batch_Size} 
    #       |\nN2R Ratio: {opt_N2R_Ratio} 
    #       |\nRegularization Term: {opt_Regularization_term} 
    #       |\nNoise Dim: {opt_Noise_Dim}
    #       |\nImpair LR: {opt_Impair_LR}
    #       |\nRepair LR: {opt_Repair_LR}")

    l1 = trial.suggest_int('l1', 32, 1024)
    l2 = trial.suggest_int('l2', 32, 1024)
    l3 = trial.suggest_int('l3', 32, 1024)
    l4 = trial.suggest_int('l4', 32, 1024)
    l5 = trial.suggest_int('l5', 32, 1024)
    l6 = trial.suggest_int('l6', 32, 1024)
    l7 = trial.suggest_int('l7', 32, 1024)
    l8 = trial.suggest_int('l8', 32, 1024)
    l9 = trial.suggest_int('l9', 32, 1024)
    n_layers = trial.suggest_int('n_layers', 1, 9)

    Layers = [l1, l2, l3, l4, l5, l6, l7, l8, l9]
    Layers = Layers[:n_layers]
    # print("Layers: ", Layers)

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    unlearned = gemu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = unlearned,
        model2 = exact,
        data_loader = valid_dl,
    )

    return div

study_gemu.optimize(objective, n_trials=3)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gemu.pkl", "wb") as fout:
    pickle.dump(study_gemu.sampler, fout)

## Standard Values

In [ ]:
# standard_model = src.gefeu._main(
#     model=None,
#     dataset_name="mnist",
#     t_Epochs=,
#     t_Batch_Size=,
#     t_Learning_Rate=,
#     t_N2R_Ratio=,
#     t_Regularization_term=,
#     t_Layers=,
#     t_Noise_Dim=,
#     t_Impair_LR=,
#     t_Repair_LR=,
#     logs=True,
#     model_eval_logs=False,
# )

___